In [1]:
import os
os.chdir('/home/icb/fatemehs.hashemig/codes/interpretable-ssl')

In [2]:
from interpretable_ssl.datasets.immune import *

In [3]:
from scarches.models.scpoli import scPoli


INFO:pytorch_lightning.utilities.seed:Global seed set to 0
/home/icb/fatemehs.hashemig/miniconda3/envs/apex-env/lib/python3.12/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/icb/fatemehs.hashemig/miniconda3/envs/apex-env/lib/python3.12/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)
 captum (see https://github.com/pytorch/captum).


In [4]:
# early_stopping_kwargs = {
#     "early_stopping_metric": "val_prototype_loss",
#     "mode": "min",
#     "threshold": 0,
#     "patience": 20,
#     "reduce_lr": True,
#     "lr_patience": 13,
#     "lr_factor": 0.1,
# }

condition_key = 'study'
cell_type_key = 'cell_type'

In [5]:
ds = ImmuneDataset()
ref, query = ds.get_train_test()

In [6]:
ref.adata

View of AnnData object with n_obs × n_vars = 29137 × 4000
    obs: 'batch', 'chemistry', 'data_type', 'dpt_pseudotime', 'cell_type', 'mt_frac', 'n_counts', 'n_genes', 'sample_ID', 'size_factors', 'species', 'study', 'tissue'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    layers: 'counts'

In [19]:
query.adata.obs.cell_type.nunique()

10

In [20]:
scpoli_model = scPoli(
    adata=ref.adata,
    condition_keys=condition_key,
    cell_type_keys=cell_type_key,
    embedding_dims=8,
    recon_loss='nb',
    labeled_indices=[]
)

/home/icb/fatemehs.hashemig/miniconda3/envs/apex-env/lib/python3.12/site-packages/scarches/models/scpoli/scpoli_model.py:143: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  self.adata.obs['conditions_combined'] = adata.obs[condition_keys]
/home/icb/fatemehs.hashemig/miniconda3/envs/apex-env/lib/python3.12/site-packages/scarches/models/scpoli/scpoli_model.py:151: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self.obs_metadata_ = adata.obs.groupby('conditions_combined').first()
/home/icb/fatemehs.hashemig/miniconda3/envs/apex-env/lib/python3.12/site-packages/scarches/models/scpoli/scpoli_model.py:156: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to reta

Embedding dictionary:
 	Num conditions: [3]
 	Embedding dim: [8]
Encoder Architecture:
	Input Layer in, out and cond: 4000 64 8
	Mean/Var Layer in/out: 64 10
Decoder Architecture:
	First Layer in, out and cond:  10 64 8
	Output Layer in/out:  64 4000 



In [21]:
from scarches.trainers.scpoli import scPoliTrainer


self = scpoli_model

trainer = scPoliTrainer(
            self.model,
            self.adata,
            labeled_indices=self.labeled_indices_,
            pretraining_epochs=10,
            condition_keys=self.condition_keys_,
            cell_type_keys=self.cell_type_keys_,
            reload_best=False,
            prototype_training=True,
            unlabeled_prototype_training=True,
        )

INFO:scarches.trainers.scpoli.trainer:GPU available: True, GPU used: True


The missing labels are: {'CD10+ B cells', 'Monocyte-derived dendritic cells', 'NK cells', 'Monocyte progenitors', 'Plasma cells', 'HSPCs', 'Erythrocytes', 'CD4+ T cells', 'CD8+ T cells', 'CD14+ Monocytes', 'CD16+ Monocytes', 'Plasmacytoid dendritic cells', 'NKT cells', 'Megakaryocyte progenitors', 'CD20+ B cells', 'Erythroid progenitors'}
Therefore integer value of those labels is set to -1
The missing labels are: {'CD4+ T cells', 'Monocyte-derived dendritic cells', 'NK cells', 'CD8+ T cells', 'CD14+ Monocytes', 'CD16+ Monocytes', 'Plasmacytoid dendritic cells', 'NKT cells', 'CD20+ B cells'}
Therefore integer value of those labels is set to -1
The missing labels are: {'CD10+ B cells', 'Monocyte-derived dendritic cells', 'NK cells', 'Monocyte progenitors', 'Plasma cells', 'HSPCs', 'Erythrocytes', 'CD4+ T cells', 'CD8+ T cells', 'CD14+ Monocytes', 'CD16+ Monocytes', 'Plasmacytoid dendritic cells', 'NKT cells', 'Megakaryocyte progenitors', 'CD20+ B cells', 'Erythroid progenitors'}
Therefo

In [22]:
trainer.clustering = 'kmeans'
trainer.n_clusters = 300

In [23]:
trainer.initialize_prototypes()


Initializing unlabeled prototypes with KMeans with a given number of300 clusters.


In [33]:
len(trainer.prototypes_unlabeled)

300

In [31]:
scpoli_model.trainer = trainer
lr, eps = 1e-3, 0.01
scpoli_model.trainer.train(5, lr, eps)

Initializing dataloaders
Starting training
 |████████████████████| 100.0%  - val_loss:  449.42 - val_cvae_loss:  449.42


In [34]:
trainer.prototypes_unlabeled

[tensor([[-0.164, -1.630, -0.205,  0.820, -0.674, -0.985,  0.662,  0.235,  0.859,
           0.404]], device='cuda:0'),
 tensor([[-2.270, -1.050,  0.602, -1.624,  0.700,  1.179,  0.374,  0.130, -0.633,
          -0.381]], device='cuda:0'),
 tensor([[-0.643, -0.516,  0.298, -1.740, -0.453,  1.193,  0.459, -0.526,  0.951,
           1.130]], device='cuda:0'),
 tensor([[ 0.652,  0.546, -0.095, -0.156,  0.647, -0.236,  0.204,  0.154, -0.262,
          -1.941]], device='cuda:0'),
 tensor([[-1.115,  0.693, -1.065, -0.106,  0.178,  0.112, -1.288,  0.744,  0.714,
           0.869]], device='cuda:0'),
 tensor([[ 0.166,  0.226, -0.024,  0.533, -1.059,  2.240, -0.586, -0.567, -0.172,
          -0.610]], device='cuda:0'),
 tensor([[-1.188, -1.295,  0.815,  1.438, -0.107,  0.436, -1.053, -0.885, -0.246,
           0.761]], device='cuda:0'),
 tensor([[-0.904, -0.488,  1.288,  2.011, -0.612,  0.332, -1.514,  0.916,  0.314,
           0.916]], device='cuda:0'),
 tensor([[ 0.469, -0.409, -0.599, -1.704

In [38]:
self.prototypes_labeled_

{'mean': None, 'cov': None}

In [37]:
scpoli_model.get_prototypes_info('unlabeled')

AssertionError: Model was trained without prototypes